# Data Preparation

In [1]:
import pandas as pd
import json
import numpy as np
import re
import seaborn as sns
import pandas.tseries
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pickle
import tensorflow as tf

## I. Reviews and Product Information for Pet Supplies

The data is imported and cleaned as follows:
1. Import reviews and product information data.
2. De-duplicate reviews and product information so there is one review for each product and reviewer.  Do the sam ething for product information data.
3. Filter to users and products with price information and enough reviews.
4. Generate numeric indices for users and products.
5. Export the cleaned data.

Step 1A. Import reviews and convert them to a Pandas dataframe.

In [2]:
with open('Data/Pet_Supplies.json', 'r') as file:
    text = file.read().split('\n')
    
reviews = []
for i in range(len(text) - 1):
    reviews.append(json.loads(text[i]))
reviews = pd.DataFrame.from_dict(reviews)
reviews.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,3.0,5,True,"09 27, 2007",A13K4OZKAAHOXS,0972585419,100indecisions,"Either my bird can't talk, or it really doesnt...",Still waiting for it to work...,1190851200,NaN,NaN
1,4.0,6,True,"08 25, 2007",A1DWYEX4P7GB7Z,0972585419,J. Weaver,The CD is a good idea for teaching a bird to s...,Feathered Phonics volume 1,1188000000,NaN,NaN
2,4.0,5,True,"02 20, 2007",A3NVN97YJSKEPC,0972585419,Theresa Ehrhardt,good cd to teach birds to talk and learn new w...,bird cd,1171929600,NaN,NaN
3,1.0,NaN,True,"12 30, 2016",A1PDMES1LYA0DP,0972585419,Kathi,I was not happy with product would like to ret...,One Star,1483056000,NaN,NaN
4,1.0,NaN,True,"12 23, 2016",AT6BH0TQLZS5X,0972585419,Alyssa,This cd is scratched and it constantly skips. ...,One Star,1482451200,NaN,NaN


Step 1B. Import product information and convert it to a Pandas dataframe.

In [3]:
with open('Data/meta_Pet_Supplies.json', 'r') as file:
    text = file.read().split('\n')
    
meta = []
for i in range(len(text) - 1):
    meta.append(json.loads(text[i]))
meta = pd.DataFrame.from_dict(meta)
meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Pet Supplies, Dogs, Health Supplies, Hip &amp...",,[Dr. Rexy hemp oil has powerful anti-inflammat...,,DR.REXY Hemp Oil for Dogs and Cats - 100% Orga...,[],,DR.REXY,[Made strictly with organic derived ingredient...,"[>#93,463 in Grocery &amp; Gourmet Food (See T...",[],Amazon Home,,,$19.90,061539972X,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Pet Supplies, Dogs]",,[Know exactly what your dog is eating with 101...,,Pet Qwerks Treat Cookbook with Cutters,[],,Pet Qwerks,"[Recipe book, Cookie cutters, Indoor/outdoor use]","190,234 in Pet Supplies (","[B075DYQ1PH, 1604334657, 1604336919, 163026043...",Pet Supplies,,,$7.86,0615553605,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Pet Supplies, Dogs, Food, Dry]",,"[, , , ]",,The Best of 101 Dog Tricks,"[0760339740, 1592533256, 1592535305, 159253730...",,,[],"76,847 in Movies &amp; TV (","[1592533256, 0760339740, 1592537308, 159253530...",Movies &amp; TV,,,,0760339597,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[, The venomous but beautiful scorpionfishes a...",,Lionfishes and Other Scorpionfishes: The Compl...,"[1911142186, 1514291983]",,Tfh/Nylabone,[Used Book in Good Condition],"435,039 in Pet Supplies (",[],Pet Supplies,,,$24.99,0793816793,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Pet Supplies, Top Selection from AmazonPets]",,[Volume 1: 96 Words &amp; Phrases! This is the...,,Pet Media Feathered Phonics The Easy Way To Te...,"[B0002FP328, B0002FP32S, B0002FP32I, B00CAMARX...",,Pet Media,[Award-winning audio CD teaches parrots how to...,"52,435 in Pet Supplies (","[B0002FP328, B00CAMARXG, B0002FP32S, B0013L6CA...",Pet Supplies,,,$6.97,0972585419,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


Step 2. Dedup reviews and product information.

In [4]:
reviews.drop_duplicates(subset = ['asin', 'reviewerID'], inplace = True)
meta.drop_duplicates(subset = ['asin'], inplace = True)

Step 3. Filter to users and products with a price and enough reviews.

In [5]:
# Rename the column names
reviews1 = reviews[['asin', 'reviewerID', 'overall', 'reviewTime', 'summary', 'vote', 'reviewText']]
reviews1.columns = ['itemId', 'userId', 'rating', 'time', 'summary', 'vote', 'reviewText']

# Merge reviews and product information
reviews2 = reviews1.merge(meta, left_on = 'itemId', right_on = 'asin')

# Filter to products with price information
reviews2 = reviews2[(reviews2['price'].notnull()) & 
                    (reviews2['price'].apply(lambda x: re.search('\\$', x) is not None))]
reviews2['price'] = reviews2['price'].apply(lambda x: float(x.split(' - ')[0].replace('$', '').replace(',', '')))

# Filter to items with at least 100 reviews
reviews_mult = reviews2.groupby('itemId')['itemId'].count() >= 100
reviews_mult = reviews_mult[reviews_mult == True]
reviews2 = reviews2[reviews2['itemId'].isin(reviews_mult.index)]

# Filter to users with at least 20 reviews
reviews_mult = reviews2.groupby('userId')['userId'].count() >= 20
reviews_mult = reviews_mult[reviews_mult == True]
reviews3 = reviews2[reviews2['userId'].isin(reviews_mult.index)]

Step 4. Generate numeric indices for users and products.  These will be used for neural networks later.

In [6]:
users = reviews3['userId'].unique()
books = reviews3['itemId'].unique()

userid2idx = {o: i for i, o in enumerate(users)}
bookid2idx = {o: i for i, o in enumerate(books)}
reviews3['userIdNumeric'] = reviews3['userId'].apply(lambda x: userid2idx[x])
reviews3['itemIdNumeric'] = reviews3['itemId'].apply(lambda x: bookid2idx[x])

<ipython-input-6-144482477e11>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews3['userIdNumeric'] = reviews3['userId'].apply(lambda x: userid2idx[x])
<ipython-input-6-144482477e11>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews3['itemIdNumeric'] = reviews3['itemId'].apply(lambda x: bookid2idx[x])


Step 5. Export the data for analysis.

In [7]:
reviews3.to_csv('Data/Data Used for Analysis/Cleaned Reviews for Pet Supplies.csv')

## II. Product Images and Information for Shoes

The data is imported and cleaned as follows:
1. Import product images and information data.
2. Filter to product information data for Shoes.
3. One-hot encode each product keyword by transposing the product information data.
4. Retrieve product images.
5. Export the cleaned data.

Step 1A. Import and combine listings data 0 to 4.

In [8]:
with open('Data/Raw Data/listings_0.json', 'r') as file:
    text1 = file.read().split('\n')
    
with open('Data/Raw Data/listings_1.json', 'r') as file:
    text2 = file.read().split('\n')
    
with open('Data/Raw Data/listings_2.json', 'r') as file:
    text3 = file.read().split('\n')
    
with open('Data/Raw Data/listings_3.json', 'r') as file:
    text4 = file.read().split('\n')

with open('Data/Raw Data/listings_4.json', 'r') as file:
    text5 = file.read().split('\n')

# Combine imported data
text = text1 + text2 + text3 + text4 + text5
text[0]

'{"brand": [{"language_tag": "nl_NL", "value": "find."}], "bullet_point": [{"language_tag": "nl_NL", "value": "Schoen in Loafer-stijl"}, {"language_tag": "nl_NL", "value": "Platform hak"}, {"language_tag": "nl_NL", "value": "Cap teen"}, {"language_tag": "nl_NL", "value": "Middenhak"}], "color": [{"language_tag": "nl_NL", "value": "Veelkleurig Vrouw Blauw"}], "item_id": "B06X9STHNG", "item_name": [{"language_tag": "nl_NL", "value": "Amazon-merk - vinden. Dames Leder Gesloten Teen Hakken,Veelkleurig Vrouw Blauw,5 UK"}], "model_name": [{"language_tag": "nl_NL", "value": "37753"}], "model_number": [{"value": "12-05-04"}], "model_year": [{"value": 2017}], "product_type": [{"value": "SHOES"}], "style": [{"language_tag": "nl_NL", "value": "Gesloten-teen pompen"}], "main_image_id": "81iZlv3bjpL", "other_image_id": ["91mIRxgziUL", "91eqBkW06wL", "A1BHZSKNbkL"], "item_keywords": [{"language_tag": "nl_NL", "value": "block heel shoes"}, {"language_tag": "nl_NL", "value": "loafer shoes"}, {"languag

Step 1B. Import product info data and convert it to a Pandas dataframe.

In [9]:
product_info = []
for i in range(len(text) - 1):
    if text[i] != '':
        text_0 = json.loads(text[i])
        try:
            for j in range(len(text_0['item_keywords'])):
                product_info.append({'item_id': text_0['item_id'], 
                                     'product_type': text_0['product_type'][0]['value'],
                                     'language': text_0['item_keywords'][0]['language_tag'],
                                     'main_image_id': text_0['main_image_id'],
                                     'item_keywords': text_0['item_keywords'][j]['value'].lower()})
        except:
            product_info.append({'item_id': text_0['item_id'], 
                                 'product_type': text_0['product_type'][0]['value'], 
                                 'language': '',
                                 'main_image_id': '',
                                 'item_keywords': ''})

product_info = pd.DataFrame.from_dict(product_info)
product_info.head()

,item_id,product_type,language,main_image_id,item_keywords
0,B06X9STHNG,SHOES,nl_NL,81iZlv3bjpL,block heel shoes
1,B06X9STHNG,SHOES,nl_NL,81iZlv3bjpL,loafer shoes
2,B06X9STHNG,SHOES,nl_NL,81iZlv3bjpL,loafers
3,B06X9STHNG,SHOES,nl_NL,81iZlv3bjpL,metallic shoes
4,B06X9STHNG,SHOES,nl_NL,81iZlv3bjpL,womens block heel shoes


Below is a distribution for individual product categories.

In [10]:
product_info['product_type'].value_counts()

CELLULAR_PHONE_CASE           2149131
SHOES                          200360
FINENECKLACEBRACELETANKLET      54334
GROCERY                         53915
HOME                            40620
                               ...   
RUG_PAD                             1
MICROWAVE_OVEN                      1
VIDEO_DEVICE                        1
SHELLFISH                           1
SKATEBOARD                          1
Name: product_type, Length: 497, dtype: int64

Step 2. Filter to product information data for Shoes.

In [11]:
product_info_shoes = product_info[(product_info['product_type'] == 'SHOES') & 
                                  (product_info['language'].apply(lambda x: x[:3]) == 'en_')]

print('Number of listings: ' + str(len(product_info_shoes['item_id'].unique())))
print('Number of unique categories: ' + str(len(product_info_shoes['item_keywords'].unique())))

Number of listings: 2462
Number of unique categories: 1091


Step 3. One-hot encode each product keyword by transposing the product information data. 

In [12]:
product_info_shoes.drop_duplicates(['item_id', 'main_image_id', 'item_keywords'], inplace = True)
product_info_shoes['value'] = 1
product_info_shoes1 = product_info_shoes.pivot(index = ['item_id', 'main_image_id'], 
                                               columns = 'item_keywords', values = 'value')

<ipython-input-12-8347df54b8b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_info_shoes.drop_duplicates(['item_id', 'main_image_id', 'item_keywords'], inplace = True)
<ipython-input-12-8347df54b8b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_info_shoes['value'] = 1


Step 4. Retrieve product images.

In [13]:
# Import the image mapping and merge to get paths to product images
images_map = pd.read_csv('Data/Raw Data/images.csv')

product_info_shoes2 = images_map.merge(product_info_shoes1, 
                                       left_on = 'image_id', right_on = 'main_image_id', how = 'right')

# Retrieve product images
reshape_imgs = []
for i in range(len(product_info_shoes2)):
    img = load_img('Data/abo-images-small/images/small/' + product_info_shoes2['path'][i])
    reshape_img = img_to_array(img) 
    reshape_img = tf.image.resize(reshape_img, (64, 64)) / 255.0
    reshape_imgs.append(reshape_img)

reshape_imgs =  tf.convert_to_tensor(reshape_imgs)

Step 5. Export the cleaned labels and product images.

In [14]:
# Export the cleaned labels
product_info_shoes2.to_csv('Data/Data Used for Analysis/Cleaned Labels for Shoes.csv')

# Export the product images
with open('Data/Data Used for Analysis/Cleaned Images.pickle', 'wb') as handle:
    pickle.dump(reshape_imgs, handle, protocol=pickle.HIGHEST_PROTOCOL)